# Change report from Direct Lake to Import Mode

This notebook can be used to:
1. Extract the BIM file of a Direct Lake file.
2. Transform the BIM File to use Import Mode.
3. Create a new Semantic Model in Import Mode.
4. Get the manual steps needed to prepare the Semantic Model for use.
5. Finalise the Semantic Model for use going forwards.
6. Bind existing reports to the new model making the old model obsolete.

In [ ]:
# Install semantic-link-labs if you haven't already

%pip install semantic-link-labs

In [ ]:
import sempy_labs as sl
import sempy_labs.report as rep

In [ ]:
# Source details
DIRECT_LAKE_MODEL_NAME = 'direct_lake_model_name'
WORKSPACE_GUID = 'cbf8bf70-4f5f-4d8a-86eb-5af279801c77'
 
# Target details
NEW_MODEL_NAME = 'import_mode_model_name'

# SQL endpoint information
ENDPOINT = 'drrjn3f71e92a3a783bdmmvui-ro3f71e92a3a785k25c8e106au.datawarehouse.fabric.microsoft.com'
LAKEHOUSE_NAME = 'lakehouse_used_by_model'

# Create an Import Version of the Semantic Model

In [ ]:
# Get the semantic model definition for the direct lake model from the source workspace
model_bim = sl.get_semantic_model_definition(
    dataset = DIRECT_LAKE_MODEL_NAME,
    format = 'TMSL',
    workspace = WORKSPACE_GUID,
    return_dataframe = False
)

In [ ]:
# Change directLake tables to import
PBI_QueryOrderList = []

for table in model_bim.get('model').get('tables'):

    first_partition = table.get('partitions')[0]

    if first_partition.get('mode') == 'directLake':
        name = table.get('name')
        entity = first_partition.get('source').get('entityName')
        schema  = first_partition.get('source').get('schemaName') or 'dbo'

        new_partition = {
            'name': name,
            'mode': 'import',
            'source': {
                'expression': [
                    'let',
                    '    Source = Sql.Database("' + ENDPOINT + '", "' + LAKEHOUSE_NAME + '"),',
                    '    navigation = Source{[Schema="' + schema + '",Item="' + entity + '"]}[Data]',
                    'in',
                    '    navigation'],
                'type': 'm'
                }
        }

        PBI_QueryOrderList.append(name)

        print(f'Updating {name} to import table')

        table.update({'partitions': [new_partition]})
                

In [ ]:
# Remove directLake annotations and expressions
annotations = model_bim.get('model').get('annotations')

for i, annotation in enumerate(annotations):

    match annotation.get('name'):

        case 'PBI_ProTooling':
            annotations.pop(i)
    
        case 'PBI_QueryOrder':
            annotations[i] = {'name': 'PBI_QueryOrder', 'value': f'{PBI_QueryOrderList}'}

model_bim.get('model').update({'annotations': annotations})

model_bim.get('model').pop('expressions')

In [ ]:
# Create the new model
sl.create_semantic_model_from_bim(
    dataset = NEW_MODEL_NAME,
    bim_file = model_bim,
    workspace = WORKSPACE_GUID
)

# Some manual steps

Changing to import seems to cause some problems with relationships and sort orders after the first refresh.

To get around this and to avoid auto date/time issues causing problems you need to download the PBIX to Desktop for the first refresh and then re-import. This will delete all relationships and sort orders, but we can recreate them and then they will work as normal.

To do this open the workspace where the Semantic Model has been created. Find the model > Download this file and open it in Power BI Desktop. In Power BI Desktop:


1. Open the Semantic Model > **File** > **Options & Setting** > **Options** > **Current File** > **Data Load**:
    1. In **Data Load** > Turn off all **Relationship settings**
    1. In **Data Load** > Turn off **Auto date/time**
1. Refresh the Power BI Model in Desktop. You will notice that existing relationships may have been deleted. This is expected. All tables should be unrelated to each other.
1. Publish the model to Power BI Service from desktop.

Then in Power BI Service:

2. Find the Semantic Model in the Workspace > open its **Settings** > Go to **Gateway and cloud connections** > Create a Connection for the SQL Endpoint.
3. Refresh the Semantic Model
4. Open the new Semantic Model > switch from **View** to **Edit** mode > Refresh in the editor

No auto-detected relationships should be created, but the refresh should be successful. This confirms that the import mode is working.

Now we need to create the Relationships and the sort order of the columns again, and bind the existing reports to the new model.


# Recreate the Relationships and Sort Orders

In [ ]:
# Get the new semantic model
new_model_bim = sl.get_semantic_model_definition(
    dataset = NEW_MODEL_NAME,
    format = 'TMSL',
    workspace = WORKSPACE_GUID,
    return_dataframe = False
)

In [ ]:
# Identify the sort by columns in the original semantic model
sort_by_columns = {}

for table in model_bim.get('model').get('tables'):

    columns = {}

    for column in table.get('columns'):
        if column.get('sortByColumn'):
            columns.update({column.get('name'): column.get('sortByColumn')})

    if columns != {}: 
        sort_by_columns.update({table.get('name'): columns})

In [ ]:
# Set the new model's sortByColumn(s) to be the Direct Lake model's sort by columns
for table in new_model_bim.get('model').get('tables'):
    
    table_name = sort_by_columns.get(table.get('name'))
    
    # If there are sort by columns to update:
    if table_name:
        for column in table.get('columns'):

            to_sort_by_column = table_name.get(column.get('name'))
            
            if to_sort_by_column:
                column.update({'sortByColumn': to_sort_by_column})
                print(f'Mapped: {column.get('name')}    ->  {to_sort_by_column}')


In [ ]:
# Set the new model's relationships to be the direct lake model's relationships
new_model_bim.get('model').update({'relationships': model_bim.get('model').get('relationships')})

# Bind Reports


In [ ]:
# Get all reports linked to the original report to bind to new model
reports = sl.list_reports_using_semantic_model(DIRECT_LAKE_MODEL_NAME, WORKSPACE_GUID)
reports

In [ ]:
# Rebind each report to the new semantic model
for report in reports['Report Id']:
    rep.report_rebind(
        report = report,
        dataset = NEW_MODEL_NAME,
        report_workspace = WORKSPACE_GUID,
        dataset_workspace = WORKSPACE_GUID
    )
